# HW03: Conditioning, Stability, and Accuracy
**Yuanxing Cheng, 810925466**

Goal: Estimate the condition number of finding the roots of polynomials.

Consider a given polynomial: 

$$p(x)= (x-1)(x-2)\cdots(x-n) \tag{1}$$

which can be writen in the form of

$$p(x) = a_1x^n + a_2 x^{n-1} + a_3 x^{n-2} + \cdots + a_{n-1} x^2 + a_n x + a_{n+1}\tag{2}$$

where $a_1, a_2, \dots, a_n, a_{n+1}$ are the coefficients of this polynomial. We know that the roots of $p(x)$ are $x = 1, 2, \dots n$.

Perturb the polynomial $p(x)$ by replacing its coefficients $a_j$ with perturbed values $\tilde{a}_j$ . Compare the roots of the perturbed polynomial:

$$\tilde{p}(x) = \tilde{a}_1x^n + \tilde{a}_2 x^{n-1} + \tilde{a}_3 x^{n-2} + \cdots + \tilde{a}_{n-1} x^2 + \tilde{a}_n x + \tilde{a}_{n+1} \tag{3}$$

The difference can be much larger than the scale of the perturbation on $a_j$, which demonstrates the ill conditioning of this problem.

## Q1
First, for a given positive integer $n$, generate the polynomial $p(x)$ in (1), by using MATLAB built-in function `poly(1:n)`. It returns a row vector $(a_1 ; a_2 ; \dots; a_n ; a_{n+1} )$, which are the
coefficients of $p(x)$, as in $(2)$.

Use MATLAB built-in function roots, to compute the roots of $p(x)$. Are those computed roots the same as the exact roots, $1, 2, \dots n$?

$Answer$

    clear
    clc

    format long

    n=input('The max order you want: \n');

    for i=1:n 
        display(['n=',int2str(i)]);
        p = poly(1:i);
        r = roots(p)
    end

So following this program you see the roots to the maximum order of polynomial you want to check. And the truth is that the computed roots start to deviate at $n=3$.

## Q2
Generate the perturbed polynomial $\tilde{p}(x)$ by perturbing each $a_j$ as $\tilde{a}_j = a_j + 10^{-10} \ast a_j \ast (\textrm{a random number})$ for $j = 1, 2, \cdots, n + 1$, $i.e.$, the relative perturbation for each coefficient is of the order $10^{-10}$. The random number can be obtained by using `rand` in MATLAB. Then use `roots` to find the roots of this perturbed polynomial $\tilde{p}(x)$.

Write a MATLAB function to plot the roots of $50$ randomly perturbed polynomials in the same graph. The $x$-axis in the plot should correspond to the **real** parts of the roots, and the $y$-axis corresponds to the imaginary parts. MATLAB functions `real` and `imag` will provide the real and imaginary parts of a complex number.

Generate four graphs, corresponding to $n = 5, 10, 15, 20$, respectively. Does the error become worse as $n$ increases?

$Answer$

    figure

    for i = 1:4
        n = 5*i;
        p = poly(1:n);

        subplot(2,2,i)
        title(['n = ' ,int2str(n)])
        hold on

        for j = 1:50     

            for k = 1:n
                d = p(k);
                e = d + 10^(-10) * d * rand;
                p(k) = e;
            end

            r = roots(p);
            X = real(r);
            Y = imag(r);

            plot(X,Y)
        end
    end

![](./MATLAB/HW04Two.png)

Obviously the error increase with larger $n$.

## Q3
Estimate the condition number of this roots-nding problem. Evaluate the relative input error by

$$relIn = \max_{j} \Bigg|\frac{\tilde{a}_j - a_j} {a_j}\Bigg|$$

and the relative output error by

$$relOut = \max_{j} \Bigg|\frac{\tilde{r}_j - r_j} {r_j}\Bigg|$$

where $\tilde{r}_j$ and ${r_j}$ represents the computed roots of $\tilde{p}(x)$ and $p(x)$, respectively. MATLAB function `max` can compute the maximum. Then the condition number can be evaluated by

$$Cond = \frac{relOut} {relIn}$$

To be more representative, for a polynomial of a given degree, $n$, you need compute this estimates for $50$ randomly perturbed polynomials and then take the average of these $50$ estimates as the final estimate for the condition number.

Plot the condition number estimate with respect to $n$, for $1 \leq n \leq 30$. You may need use log scale in the y-axis to catch the behavior of this plot (see MATLAB function `semilogy`). Explain what you observe in this plot and the plots you obtained in part 2.

$Answer$

    clear
    clc

    m = 30;
    relIn = zeros(1,m);
    relOut = zeros(1,m);l

    for i = 1:m
        p = poly(1:i);

        sum = 0;

        for j = 1:50

            tilde_p = p + 10^(-10) * p .* rand(1,i+1);
            In = abs( (tilde_p - p)./p );       

            r = roots(p);
            tilde_r = roots(tilde_p);
            Out = abs( (tilde_r - r)./r );
        end

        %Average operation will be cancled when calculating the conditional number
        relIn(i) = max(In);
        relOut(i) = max(Out);
    end

    Cond = relOut ./ relIn;

    figure

    semilogy(1:m, Cond);
    axis([0 31 10^(-2) 10^(12)])
    grid on

    Y = ylabel({'\fontname{Courier New} \fontsize{16} \bf Condition';'\fontname{Courier New} \fontsize{16} \bf Number'}, 'Rotation',0);
    X = xlabel('\fontname{Courier New} \fontsize{16} \bf n', 'FontSize',16);

    set(Y, 'Units', 'Normalized', 'Position', [-0.11, 0.5, 0]);
    set(X, 'Units', 'Normalized', 'Position', [0.5, -0.03, 0]);

![](./MATLAB/HW04Three.png)

So in part 2, there're four plots and the error can be seen with $n = 15, 20$. So with the degree of the polynomial going up, the root-finding by MATLAB is getting more and more incorrect. There're shouldn't be any complex number but seen from the figure, there "is" after $n$ gets larger like $15$ or more. And we can also find that at $n = 20$, even the real part of the root is incorrect because the real root is only integer from $1$ to $20$, but there's a $21$ in the axis. So another conclusion maybe that larger $n$ leads to a worse result.

Now in part 3, we estimate the condition number of this root-finding. It keeps going up with increasing $n$, from $10^{-1}$ to $10^{10}$. So it's getting more and more ill-conditioned.

And besides I also tried to make a experiment with larger $n$ like $100$, and I find that the conditional number will not increase anymore, so it seems that its upper limit is $10^{10}$. Can't figure out why.

$Explanation$

The relative error in the output are about $1$, and the relative error in the input is about the inverse of pertubation, which is in the order of $10^{10}$. And that's way the condition number will not going up even the n gets bigger.